# <h1 align="center">Modelo para la predicción de cancelaciones</h1>


#### [Enlace al conjunto de datos (kaggle)](https://www.kaggle.com/jessemostipak/hotel-booking-demand)

   ### Índice:
   
       1. Importación de librerías y datos

       2. Análisis preliminar
       
       3. Preprocesado
       
       4. EDA
       
       5. Gráfico de correlaciones
       
       6. Exploración y transformación de variables
       
       7. Construcción del modelo



## 1. Importación de librerías y datos

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
!pip install sorted-months-weekdays
!pip install pip install sort-dataframeby-monthorweek
import sort_dataframeby_monthorweek as sd
import datetime
from sklearn.metrics import mean_squared_error
!pip install pmdarima
from pmdarima import auto_arima
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
import pickle

import folium
from folium.plugins import HeatMap
import plotly.express as px
        
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv("../input/hotel-booking-demand/hotel_bookings.csv")

## 2. Análisis preliminar

In [ ]:
data.head()

Nos encontramos con un dataset con un total de 32 columnas.

In [ ]:
data.info()

## 3. Preprocesado de los datos

Pasos a seguir:

- Comprobación de la correcta tipología y rol de las variables
- Tratamiento de datos faltantes
- Corrección preliminar de errores detectados
- Búsqueda de datos atípicos

#### 3.1.Tratamiento de datos faltantes: análisis de los valores NA
Las variables country, agent y company destacan por su número de NA´S

In [ ]:
NaN = data.isna().sum()
NaN

#### Eliminación de las dos variables con gran cantidad de NA'S

In [ ]:
data = data.drop(['company', 'agent'], axis=1)

Al considerar las diferentes opciones de tratamiento de NA's nos decantamos por no imputar estos valores con sus medias ni 0's dado que para el caso de la variable 'company', los missing values representaban un 96 \% del total de la data de esa columna. Por lo tanto, modificarlos supondría una grave alteración de los datos.
Hemos de tener además en cuenta que la eliminación de variables sólo debe llevarse a cabo cuando la proporción de missings sea muy elevada (superior al 50%) y, por tanto, la pérdida de información no lo sea tanto.

Para el caso de ‘agent’, aunque el porcentaje no era tan elevado, de igual manera decidimos eliminarlo dado que no nos aportaba mucha información para el modelo.



#### Ahora se eliminan las observaciones de las variables en las que hay pocos NA´s
 

In [ ]:
data = data.dropna(subset=['country', 'children', 'arrival_date_week_number'], axis=0) 

data = data.reset_index(drop=True)
data.head()

#### Nuevo análisis de los NA´S

In [ ]:
NaN = data.isna().sum()
NaN

#### 3.2. Corrección preliminar de errores detectados:
Al analizar las características de las reservas, en concreto en lo que se refiere a los huéspedes, nos damos cuenta de que existen un total de 170 observaciones que cumplen con la condición de que: (data.children == 0) & (data.adults == 0) & (data.babies == 0)

No puede haber 0's en una misma observación en adults, children y babies (pues no se puede hacer una reserva sin huéspedes). 


In [ ]:
filter = (data.children == 0) & (data.adults == 0) & (data.babies == 0)
sum(filter)

#### Se concluye que se trata de un error, por lo que procedemos a eliminarlos:

In [ ]:
data = data[~filter]
data.head()

#### 3.3 Búsqueda de datos atípicos

Esta primera corrección de registros conflictivos nos anima a proceder con la búsqueda de datos atípicos, esto es, observaciones numéricamente distantes del resto de los datos.

Resulta de gran importancia poder lidiar con este tipo dada su gran influencia en los resultados, si los modelos no son robustos.
Para la detección de estos datos, dado que no existe un método perfecto de detección, hemos hecho uso de varios, considerando como atípicas aquellas observaciones consideradas como tal por más de un método.


In [ ]:
data.describe()

Comenzamos con la detección de outliers visualizando los boxplot de las diferentes variables que conforman nuestro modelo. De su visualización obtenemos un total de 8 variables que presentan cierta problemática: 'lead time', 'stays in weekend nights', 'stays in week nights', 'adults', 'babies', 'required car parking spaces', 'adr', 'previous cancellations'.


In [ ]:
columnas = ['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 
           'babies', 'required_car_parking_spaces', 'adr', 'previous_cancellations']
n = 1
plt.figure(figsize=(20,15))

for column in columnas:
  plt.subplot(4,4,n)
  n = n+2
  sns.boxplot(data[column])
  plt.tight_layout()

Verificamos esta información que nos dan los boxplot empleando técnicas de clustering, en concreto, el algoritmo DBScan. Se utiliza como método de detección de anomalías basado en la densidad de datos, tanto unidimensionales como multidimensionales.
Distingue entre diferentes puntos dentro del clúster: core points, border points y noise points. Los llamados "noise points" son aquellos que no pertenecen a ningún clúster, y que por lo tanto pueden llegar a considerarse anómalos.


In [ ]:
from sklearn.cluster import DBSCAN

columnas = ['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 
           'babies', 'required_car_parking_spaces', 'adr', 'previous_cancellations']

In [ ]:
deteccion_outliers = DBSCAN(min_samples = 2, eps = 3)
clusters = deteccion_outliers.fit_predict(data[columnas])
list(clusters).count(-1)

En nuestro caso, por ejemplo, encontramos que la variable “lead_time” tiene un total de 2 noisy points. En total, esas 8 variables que veíamos problemáticas suman 2713 noisy points.


#### Se procede a sustituir la mayoría de los valores atípicos por otros dentro del último cuartil o por el valor cero dependiendo del caso.

In [ ]:
data.loc[data.lead_time > 400, 'lead_time'] = 400
data.loc[data.stays_in_weekend_nights >=  5, 'stays_in_weekend_nights'] = 5
data.loc[data.stays_in_week_nights > 20, 'stays_in_week_nights'] = 20
data.loc[data.adults > 10, 'adults'] = 10
data.loc[data.babies > 8, 'babies'] = 0
data.loc[data.required_car_parking_spaces > 5, 'required_car_parking_spaces'] = 0
data.loc[data.adr > 1000, 'adr'] = 1000
data.loc[data.adr < 30, 'adr'] = 0

#### Comprobación de que no hay registros que sumen cero y por tanto el numero de registros total es correcto:

In [ ]:
# TOTAL DE HUÉSPEDES

data['Total_Guests'] = data['adults'] + data['children']

# Comprobamos que efectivamente no hay ningún registro que sume 0!

filter = data.Total_Guests != 0
sum(filter) # el número de registros total son 118728, asi que es correcto


------------------------------------

## 4. EDA

En este apartado, hemos abordado el análisis de diferentes preguntas capaces de darnos mucha información, no solo a la hora de construir nuestro modelo de predicción, sino de entender un poco más el mundo de las reservas de hoteles, así como las necesidades que les pueden surgir a las empresas y que nosotros tratamos de solucionar con nuestras herramientas de Big Data.



En este primer análisis exploratorio se realiza un recuento del número de huéspedes por nacionalidad.
Destaca Portugal con una clara diferencia respecto Gran Bretaña.

#### 4.a. Países con más visitas

In [ ]:
paises_mas_visitas = data[data['is_canceled'] == 0]['country'].value_counts().reset_index()
paises_mas_visitas.columns = ['country', 'No of guests']
paises_mas_visitas

#### Clara primacía de Europa y en especial de Portugal

In [ ]:
basemap = folium.Map()
guests_map = px.choropleth(paises_mas_visitas, locations = paises_mas_visitas['country'],color_continuous_scale="portland",
                           color = paises_mas_visitas['No of guests'], hover_name = paises_mas_visitas['country'])
guests_map.show()


#### 4.b. ¿Cuánto se paga por una noche de alojamiento?

In [ ]:
cuanto_se_paga = data[data['is_canceled'] == 0] 

# Vamos a usar para este fin la variable 'adr' (Average Daily Rate as defined by dividing the sum of all lodging transactions by the total number of staying nights)

px.box(data_frame = cuanto_se_paga, x = 'reserved_room_type', y = 'adr', color = 'hotel')

#### Ahora se visualiza sin diferenciar

In [ ]:
px.box(data_frame = cuanto_se_paga, x = 'reserved_room_type', y = 'adr')

#### 4.c. ¿Varía el precio según la época del año?

In [ ]:
cuanto_se_paga_MES = cuanto_se_paga.groupby(['arrival_date_month'])['adr'].mean().reset_index()
cuanto_se_paga_MES

#### 4.d. Diferencias por el tipo de alojamiento:

In [ ]:
# En primer lugar, vamos a crear una función heciendo uso de la librería sort_dataframeby_monthorweek y su función Sort_Dataframeby_Month


def group_by_month(df, column_name):
    return sd.Sort_Dataframeby_Month(df, column_name)

data_resort = data[(data['hotel'] == 'Resort Hotel') & (data['is_canceled'] == 0)]

data_city = data[(data['hotel'] == 'City Hotel') & (data['is_canceled'] == 0)]

resort_hotel = data_resort.groupby(['arrival_date_month'])['adr'].mean().reset_index()

city_hotel=data_city.groupby(['arrival_date_month'])['adr'].mean().reset_index()

final_hotel = resort_hotel.merge(city_hotel, on = 'arrival_date_month')

final_hotel.columns = ['mes', 'precio_por_resort', 'precio_por_hotel_ciudad']

final_prices = group_by_month(final_hotel, 'mes')
final_prices

#### 4.e. ¿Existe alguna relación entre el número de días transcurridos desde la reserva, y las cancelaciones?

In [ ]:
# lead_time: nº días transcurridos entre la fecha de reserva y el dia de llegada. ANTICIPACIÓN DE LA RESERVA.

plt.figure(figsize=(12,6))
sns.barplot(x='arrival_date_year', y='lead_time',hue='is_canceled', data= data)

plt.title('Arriving year, Leadtime and Cancelations')

# En el gráfico de abajo, podemos observar que en los 3 años de análisis, las reservas con más de 100 días transcurridos
# tienen mayor posibilidad de ser canceladas.

Hay una tendencia al alza, cuantos más días transcurren más cancelaciones se producen.

#### 4.f. ¿Se distribuyen de forma homogénea las llegadas dependiendo del mes?


In [ ]:
plt.figure(figsize=(15,6))

sns.countplot(data = data, x = 'arrival_date_day_of_month', hue='hotel')
plt.show()

Se concluye que se distribuyen de forma razonablemente homogenea. El valor más bajo se registra los días 31, esto se debe a que no todos los meses tienen 31 días y por tanto el recuento de llegadas es inferior.

#### 4.g. ¿Tienen mayor índice de cancelación los fines de semana?

In [ ]:
plt.figure(figsize=(15, 8))

plt.subplot(1, 2, 2)
sns.countplot(data = data, x = 'stays_in_weekend_nights', hue='is_canceled')
plt.title('Número de noches en fin de semana', size=11)
plt.subplots_adjust(right=2)

plt.show()


# Como podemos observar abajo, la mayoría de las reservas para fin de semana no se han cancelado. 
# INFORMACIÓN ÚTIL PARA PREDECIR.

El gráfico muestra el número de noches en fin de semana que tienen las reservas. Si se suman los valores de las reservas con una o dos noches se ve claramente como el valor de estas es superior al del resto de la semana (reservas con valor cero en el número de noches) por lo que hay un claro aumento de la reservas en fin de semana. Además el número de reservas canceladas con alguna noche en fin de semana es proporcionalmente inferior al del resto de reservas.

#### 4.h. ¿Qué tipo de régimen de pensión  eligen los huéspedes?

In [ ]:
# BB: BED AND BREAKFAST
# FB: FULL BOARD (PENSIÓN COMPLETA)

meal_labels= ['BB','HB', 'SC', 'Sin definir', 'FB']
size = data['meal'].value_counts()
plt.figure(figsize=(8,8))
cmap =plt.get_cmap("Pastel2")
colors = cmap(np.arange(6)*1)
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(size, labels=meal_labels, colors=colors, wedgeprops = { 'linewidth' : 3, 'edgecolor' : 'white' })
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('Tipo de régimen', weight='bold')
plt.show()

# Como podemos observar en el gráfico de abajo, la gran mayoría de reservas son con régimen de Bed & Breakfast
# Casi nadie elige Full Board

La mayoria de las reservas son con régimen de Bed & Breakfast y sólo una parte muy pequeña elije pensión completa.

#### 4.i. Pesos en función del tipo de alojamiento:

In [ ]:
group_meal_data = data.groupby(['hotel','meal']).size().unstack(fill_value=0).transform(lambda x: x/x.sum())
group_meal_data.applymap('{:.2f}'.format)

El 94% de las reservas con pensión completa (full Board) se hacen en los Resort Hotel

#### 4.j. Datos de interés sobre España

In [ ]:
deposit_type_Spain = data[data['country'] == "ESP"]['deposit_type'].value_counts().reset_index()
deposit_type_Spain.columns = ['Tipo de depósito', 'Nº Reservas']
deposit_type_Spain

In [ ]:
reservation_date_Spain = data[data['country'] == "PRT"][data['is_canceled'] == 0]['arrival_date_year'].value_counts().reset_index()
reservation_date_Spain.columns = ['Año', 'Nº Reservas']
reservation_date_Spain

La mayoría de las reservas se realizaron sin depósito y las que se realizaron con deposito sólo una parte ínfima fue con opción a reembolso. por otro lado, el número anual de reservas tiene una tendencia decreciente.

#### 4.k. Relación entre total_of_special_requests, previous_cancellations y cancelaciones

In [ ]:
sns.countplot(data=data, x= 'total_of_special_requests', hue='is_canceled')

Casi la mitad de las reservas sin solicitudes especiales se han cancelado.
Sin embargo, vemos que en cuanto hay una solicitud de peticiones especiales o más, hay muchas mas reservas que no se cancelan que se cancelan.



In [ ]:
sns.countplot(data=data, x= 'previous_cancellations', hue='is_canceled')

In [ ]:
sns.countplot(data=data, x= 'booking_changes', hue='is_canceled')

In [ ]:

# Analizamos la variable "lead time" en relación con la cancelación

# Primero, vamos a dividir "lead time" en varios intervalos: menos de 100 dias, de entre 100-355 dias y 365 días o mas

lead_time_1 = data[data["lead_time"] < 100]
lead_time_2 = data[(data["lead_time"] >= 100) & (data["lead_time"] < 365)]
lead_time_3 = data[data["lead_time"] >= 365]


# A continuación calculamos las cancelaciones de acuerdo a estos grupos: 

lead_cancel_1 = lead_time_1["is_canceled"].value_counts()
lead_cancel_2 = lead_time_2["is_canceled"].value_counts()
lead_cancel_3 = lead_time_3["is_canceled"].value_counts()



total_lead_days_cancel = pd.DataFrame(data=[lead_cancel_1,lead_cancel_2,lead_cancel_3],
             index=["[0,100) days", "[100,365) days", "[365,max) days"])
# Mostramos la tabla
total_lead_days_cancel



In [ ]:
autopct='%1.1f%%'

In [ ]:
# Gráficamente lo representamos así:

fig, ax = plt.subplots(1,3, figsize=(15,4))
ax[0].pie(np.array([total_lead_days_cancel[0][0], total_lead_days_cancel[1][0]]),
          labels=["not_canceled", "canceled"], autopct='%1.1f%%', startangle=90)
ax[0].set_title("lead_time entre 0 y 100 dias", size=15)
ax[1].pie(np.array([total_lead_days_cancel[0][1], total_lead_days_cancel[1][1]]),
          labels=["not_canceled", "canceled"], autopct='%1.1f%%', startangle=90)
ax[1].set_title("lead_time entre 100 y 365 dias", size=15)
ax[2].pie(np.array([total_lead_days_cancel[0][2], total_lead_days_cancel[1][2]]),
          labels=["not_canceled", "canceled"],autopct='%1.1f%%', startangle=90)
ax[2].set_title("lead_time 365 dias o mas", size=15)
plt.tight_layout()
plt.show()

De este gráfico extraemos que cuando el tiempo de espera es mayor, aumentan las probabilidades de cancelación. La cantidad de reservas se mantiene estable en general entre 20 y 100 días y a continuación desciende.


----------------------------------------------------------------------------------------------------


De este Análisis Exploratorio de los datos nos vamos a quedar con la siguiente información útil para el modelo:

- La indudable importancia de la variable hotel, referida al tipo de hotel que se está reservando, pues como hemos podido ver en varios gráficos, la diferencia en la interpretación es considerable cuando habalmos de un resort frente a cuando hablamos de un hotel de ciudad.



- Por otro lado, las peculiaridades que ciertas variables como “lead_time”, total number of guests, “Booking_changes” y “previous_cancellations”, las cuales consideramos que pueden tener un peso considerable para predecir de una manera más precisa futuras cancelaciones. 

----------------------------


## 5. Gráfico de correlaciones


In [ ]:
plt.figure(figsize = (24, 12))

corr = data.corr()
sns.heatmap(corr, annot = True, linewidths = 1)
plt.show()


Pasamos a anlizar el gráfico de correlaciones,que nos va a dar una información muy útil a la hora de definir nuestro modelo.
Como podemos analizar, las correlaciones más altas con la variable "is_cancelled" (nuestra target) son:
"lead_time" (0.29), "total_of_special_requests" (-0.24), "booking_changes" (-0.15)y "previous_cancellations" (0.11).
Del correlograma extraemos lagran cantidad de valores que guardan una correlación negativa con la variable objetivo. Las correlaciones por lo general son bajas.

Que la variable que más correlación tenga sea lead_time no nos soprende en el sentido que reservas hechas con una menor anticipación están menos afectadas por imprevistos que puedan desembocar en su posterior cancelación.

Como es razonable, la variable Total_Guests que hemos creado mediante feature engineering, está muy correlacionada con adults, children e incluso babies, por lo que se procederá a eliminar estas últimas. 

La correlación entre las variables total_guests y adr (0.43) se debe a que a mayor número de personas por reserva, mayor es el precio de la misma.


Stays_in_weekend_night / Stays_in_week_night: Las reservas con noches de diario y fines de
semana tienen más probabilidad de ser canceladas. Las estancias en días de diario son habituales
en viajes de negocios y las de fin de semana son más cortas y suelen surgir menos imprevistos.
Correlación (0.46).

La correlación entre las variables total_guests / adr (0.43) se debe a que, a mayor número de
personas por reserva, mayor es el precio de la misma.

Repeated Guests / Previous_bookings_not_canceled: si un cliente repite y previamente
realizó reservas las cuales no canceló (0.43) significa que será más propenso a cancelar por
cambios en sus preferencias.

Special_request / adr las camas extra, determinados servicios u otras comodidades hacen que
los clientes cambien de opción con más facilidad por diferencias de precio (0.18).

Total_guests / Special_request: a mayor número de personas por reserva y peticiones
especiales mayor riegos de cancelación. Puede deberse a discrepancias entre las personas de la
reserva. Correlación (0.16).

Previous_cancelations / Previous_bookings_not_
cancelled:  Los clientes que previamente han reservado y han cancelado algunas de esas
reservas tienen más probabilidad de cancelar sus nuevas reservas. (0.15).

Parking / lead_time (0.12) el tiempo entre la reserva y el book-in es determinante. Puede ser
ocasionado por cambios en la planificación del viaje e imprevistos entre otros factores.

## 6. Exploración y transformación de variables

------------------------------------


#### 6.1 Eliminamos algunas columnas no útiles

In [ ]:

eliminar = ['days_in_waiting_list', 'arrival_date_year', 'arrival_date_year', 'assigned_room_type',
            'reservation_status', 'country', 'days_in_waiting_list', 'children', 'adults', 'babies']

data.drop(eliminar, axis = 1, inplace = True)

-----------------------------------------------------------------------------------------------------------------

#### 6.2. Diferenciación entre categóricas y numéricas

In [ ]:
cat_cols = [col for col in data.columns if data[col].dtype == 'O']
data[cat_cols].head()
cat_df = data[cat_cols]

In [ ]:
cat_df['reservation_status_date'] = pd.to_datetime(cat_df['reservation_status_date'])

cat_df['year'] = cat_df['reservation_status_date'].dt.year
cat_df['month'] = cat_df['reservation_status_date'].dt.month
cat_df['day'] = cat_df['reservation_status_date'].dt.day

# Aquí podríamos eliminar la variable a partir de la cual hemos sacado la información pero preferimos conservarla de momento.
# Al final lo acabamos haciendo, junto con otras columnas que no vemos útiles.

cat_df.drop(['reservation_status_date','arrival_date_month'] , axis = 1, inplace = True)



In [ ]:
for col in cat_df.columns:
    print(f"{col}: \n{cat_df[col].unique()}\n")

In [ ]:
cat_df.head()

#### 6.3. Categorización para facilitar los cálculos:

In [ ]:
cat_df['hotel'] = cat_df['hotel'].map({'Resort Hotel' : 0, 'City Hotel' : 1})

cat_df['meal'] = cat_df['meal'].map({'BB' : 0, 'FB': 1, 'HB': 2, 'SC': 3, 'Undefined': 4})

cat_df['market_segment'] = cat_df['market_segment'].map({'Direct': 0, 'Corporate': 1, 'Online TA': 2, 'Offline TA/TO': 3,
                                                           'Complementary': 4, 'Groups': 5, 'Undefined': 6, 'Aviation': 7})

cat_df['distribution_channel'] = cat_df['distribution_channel'].map({'Direct': 0, 'Corporate': 1, 'TA/TO': 2, 'Undefined': 3,
                                                                       'GDS': 4})

cat_df['reserved_room_type'] = cat_df['reserved_room_type'].map({'C': 0, 'A': 1, 'D': 2, 'E': 3, 'G': 4, 'F': 5, 'H': 6,
                                                                   'L': 7, 'B': 8})

cat_df['customer_type'] = cat_df['customer_type'].map({'Transient': 0, 'Contract': 1, 'Transient-Party': 2, 'Group': 3})


cat_df['deposit_type'] = cat_df['deposit_type'].map({'No Deposit': 0, 'Refundable': 1, 'Non Refund': 2})

In [ ]:
num_df = data.drop(columns = cat_cols, axis = 1)
num_df.drop('is_canceled', axis = 1, inplace = True)
num_df

#### 6.4. Normalizamos las variables numéricas

In [ ]:
num_df['lead_time'] = np.log(num_df['lead_time'] + 1)
num_df['arrival_date_week_number'] = np.log(num_df['arrival_date_week_number'] + 1)
num_df['arrival_date_day_of_month'] = np.log(num_df['arrival_date_day_of_month'] + 1)
num_df['adr'] = np.log(num_df['adr'] + 1)

# eliminamos los nas que nos genera esa normalizacion
num_df['adr'] = num_df['adr'].fillna(value = num_df['adr'].mean())


-------------------------------------------------------


----------------------------------------------------------------------------

## 7. Construcción del modelo

In [ ]:
X = pd.concat([cat_df, num_df], axis = 1)

y = data['is_canceled']
X.shape, y.shape

In [ ]:
X.head()

####  Separamos en Train y en Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

In [ ]:
X_train.head()

--------------------------------------------------------------------------------------------------------------

#### 7.1. Modelo 1: Regresión Logística

Para la construcción de nuestro modelo, pondremos nuestro foco en el tipo de variable objetivo que tenemos. Al tratarse de una variable binaria ( 1=se ha cancelado y 0=no se ha cancelado), elegiremos un algoritmo de regresión logística. Este modelo nos devolverá la probabilidad de cancelación, que posteriormente se implementará en nuestra aplicación.

In [ ]:
X_test.dtypes

In [ ]:
lr = LogisticRegression(solver='liblinear')
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

acc_lr = accuracy_score(y_test, y_pred_lr)
conf = confusion_matrix(y_test, y_pred_lr)
clf_report = classification_report(y_test, y_pred_lr)

print(f"Accuracy Score of Logistic Regression is : {acc_lr}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

Como podemos observar más arriba, nuestro modelo tiene un scoring de 0.807. En cuanto a la matriz de confusión, la diagonal principal representará los casos en los que nuestro modelo ha acertado. La otra diagonal, por el contrario, representará los casos en los que nuestro modelo ha fallado. En concreto, nuestro modelo tiene 21.072 verdaderos positivos, 1.157 falsos positivos, 5.695 falsos negativos y 7.695 verdaderos negativos. 

#### 7.2. Modelo 2: XGBoost

In [ ]:
xgb = XGBClassifier(booster = 'gbtree', learning_rate = 0.1, max_depth = 5, n_estimators = 180)
xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
conf = confusion_matrix(y_test, y_pred_xgb)
clf_report = classification_report(y_test, y_pred_xgb)

print(f"Accuracy Score of Ada Boost Classifier is : {acc_xgb}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

#### 7.3. Conclusión e implementación del modelo en la aplicación

------------------------------------------------------------------------------------------------------------

Con este modelo, empleando un clasificador XGBoost con learning_rate = 0.1, max_depth = 5, n_estimators = 180 se ha conseguido un accuracy de 0.982.

Sin embargo, no se puede dejar de lado el modo en el que se va a implementar este modelo.
Hemos optado por el diseño de una interfaz sencilla, amigable para los usuarios de la empresa, por lo que parándonos a analizar las variables que han sido empleadas en el modelo consideramos la posibilidad de reducir nuestra dimensionalidad.

Para ello, se empleará el método de lo particular a lo general analizando como se comporta el accuracy y hasta qué punto somos capaces de simplificar nuestro modelo.



In [ ]:
X.info()

A continuación, se muestra a modo de ejemplo el procedimiento que seguimos. Hemos ido incluyendo una por una las diferentes variables hasta que hemos obtenido un accuracy satisfactorio, logrando reducir la dimensionalidad de los datos.

Comenzamos incluyendo únicamente las variables "hotel" y "meal", y analizando las predicciones.

In [ ]:
X_2=X[["hotel","meal"]]
y = data['is_canceled']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_2, y, test_size = 0.30)


xgb = XGBClassifier(booster = 'gbtree', learning_rate = 0.1, max_depth = 5, n_estimators = 180)
xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
conf = confusion_matrix(y_test, y_pred_xgb)
clf_report = classification_report(y_test, y_pred_xgb)

print(f"Accuracy Score of Ada Boost Classifier is : {acc_xgb}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

Obtenemos inicialmente un accuracy de 0.627.

Continuamos incluyendo una a una las variables hasta que obtenemos el siguiente modelo:

In [ ]:
X_3=X[["hotel","meal", "reserved_room_type", "customer_type","year", "month", "day", "lead_time", "deposit_type",
       "arrival_date_week_number",
       "arrival_date_day_of_month",      
       "stays_in_weekend_nights", "stays_in_week_nights","is_repeated_guest", 
       "previous_cancellations","previous_bookings_not_canceled",
       "adr","required_car_parking_spaces","total_of_special_requests", "Total_Guests", "booking_changes"]]
y = data['is_canceled']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_3, y, test_size = 0.30)


xgb = XGBClassifier(booster = 'gbtree', learning_rate = 0.1, max_depth = 5, n_estimators = 180)
xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
conf = confusion_matrix(y_test, y_pred_xgb)
clf_report = classification_report(y_test, y_pred_xgb)

print(f"Accuracy Score of Ada Boost Classifier is : {acc_xgb}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

Prácticamente no varía el accuracy

Nos quedamos con las siguientes variables: 

"hotel"
"meal"
"reserved_room_type"
"customer_type"
"year"
"month"
"day"
"deposit_type"                    
"lead_time"
"arrival_date_week_number"
"arrival_date_day_of_month"
"stays_in_weekend_nights"
"stays_in_week_nights"
"is_repeated_guest"
"previous_cancellations"
"previous_bookings_not_canceled"
"adr"
"required_car_parking_spaces"
"total_of_special_requests"
"Total_Guests"
"Booking_changes"


Hemos prescindido de:

"market segment"
"distribution_channel"
"num_personas"

#### **Importancia de las variables en el modelo**

In [ ]:
corr = abs(data.corr())
corr[['is_canceled']].sort_values(by = 'is_canceled',ascending = False).style.background_gradient()

**Lead_time** es la que mayor correlación tiene. Tal como se mencionó en la explicación del gráfico de correlaciones.

**Peticiones especiales** aumentan la probabilidad de cancelación, se entiende que las camas extra, determinados servicios u otras comodidades hacen que los clientes cambien de opción con más facilidad por diferencias de precio, calidades o cambios en sus necesidades. Tiene una correlación de 0.18 con la variable adr (precio de la reserva)

**Parkings:** la correlación mas importante la tiene con lead_time (0.12) por lo que nuevamente el tiempo entre la reserva y el book-in es determinante. Puede ser ocasionado por cambios en la planificación del viaje e imprevistos entre otros factores.

**Cambios en la reserva:** El cliente tiene cambios en sus necesidades y al modificarla, también busca otras opciones las cuales podrían interesarle más.

**Cancelaciones previas:** A mayor número de cancelaciones previas realizadas por el cliente anteriores a la reserva actual, mayor probabilidad de cancelación de reserva.

**Repeated Guests:** si es un cliente que repite reserva, es probable que cancele debido a que puede buscar un destino nuevo a su vez.


#### Se graba el modelo en el directorio para la posterior construcción de la app

In [ ]:
# Se graba el modelo en el directorio (descomentar si se quiere guardar)
"""
pkl_filename = "model_prediccion_cancelaciones.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(xgb, file)
"""

In [ ]:
# Se carga el modelo para comprobar que funciona (descomentar si se quiere comprobar una vez guardado el modelo)
"""
pkl_filename = "model_prediccion_cancelaciones.pkl"
with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)
"""

#### Desempeño del modelo

In [ ]:
# Descomentar si se han ejecutado las anteriores celdas (grabación y carga del modelo)
"""
score = model.score(X_test, y_test)
print(score)
"""